In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open(r"C:\Users\Pratyush\Downloads\intents.json").read()
intents = json.loads(intents_file)

In [6]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)      
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [9]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [10]:
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


In [11]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200
47/47 [==============================] - 0s 5ms/step - loss: 2.2410 - accuracy: 0.0851
Epoch 2/200
47/47 [==============================] - 0s 680us/step - loss: 2.2332 - accuracy: 0.1277
Epoch 3/200
47/47 [==============================] - 0s 595us/step - loss: 2.0952 - accuracy: 0.2128
Epoch 4/200
47/47 [==============================] - 0s 557us/step - loss: 2.0266 - accuracy: 0.3617
Epoch 5/200
47/47 [==============================] - 0s 595us/step - loss: 1.9399 - accuracy: 0.3191
Epoch 6/200
47/47 [==============================] - 0s 510us/step - loss: 1.7483 - accuracy: 0.4681
Epoch 7/200
47/47 [==============================] - 0s 425us/step - loss: 1.6898 - accuracy: 0.5319
Epoch 8/200
47/47 [==============================] - 0s 424us/step - loss: 1.4165 - accuracy: 0.5106
Epoch 9/200
47/47 [==============================] - 0s 425us/step - loss: 1.3446 - accuracy: 0.5106
Epoch 10/200
47/47 [==============================] - 0s 449us/step - loss: 1.3457 - accuracy

47/47 [==============================] - 0s 537us/step - loss: 0.1615 - accuracy: 0.9574
Epoch 82/200
47/47 [==============================] - 0s 510us/step - loss: 0.0444 - accuracy: 1.0000
Epoch 83/200
47/47 [==============================] - 0s 510us/step - loss: 0.0535 - accuracy: 1.0000
Epoch 84/200
47/47 [==============================] - 0s 595us/step - loss: 0.0302 - accuracy: 1.0000
Epoch 85/200
47/47 [==============================] - 0s 595us/step - loss: 0.0901 - accuracy: 0.9787
Epoch 86/200
47/47 [==============================] - 0s 425us/step - loss: 0.0302 - accuracy: 1.0000
Epoch 87/200
47/47 [==============================] - 0s 425us/step - loss: 0.0426 - accuracy: 1.0000
Epoch 88/200
47/47 [==============================] - 0s 425us/step - loss: 0.0306 - accuracy: 0.9787
Epoch 89/200
47/47 [==============================] - 0s 425us/step - loss: 0.0453 - accuracy: 1.0000
Epoch 90/200
47/47 [==============================] - 0s 510us/step - loss: 0.0624 - accuracy: 

47/47 [==============================] - 0s 692us/step - loss: 0.0223 - accuracy: 1.0000
Epoch 161/200
47/47 [==============================] - 0s 595us/step - loss: 0.0318 - accuracy: 1.0000
Epoch 162/200
47/47 [==============================] - 0s 510us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 163/200
47/47 [==============================] - 0s 510us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 164/200
47/47 [==============================] - 0s 595us/step - loss: 0.0073 - accuracy: 1.0000
Epoch 165/200
47/47 [==============================] - 0s 510us/step - loss: 0.0147 - accuracy: 1.0000
Epoch 166/200
47/47 [==============================] - 0s 595us/step - loss: 0.0106 - accuracy: 1.0000
Epoch 167/200
47/47 [==============================] - 0s 510us/step - loss: 0.0289 - accuracy: 1.0000
Epoch 168/200
47/47 [==============================] - 0s 605us/step - loss: 0.0315 - accuracy: 1.0000
Epoch 169/200
47/47 [==============================] - 0s 534us/step - loss: 0.0159 - a

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open(r"C:\Users\Pratyush\Downloads\intents.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence

def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)

    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

    # sorting strength probabilit
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI


In [41]:
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="Red", font=("Verdana", 12 )) 
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Bot: " + res + '\n\n')         
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)
root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatBox = Text(root, bd=0, bg="papaya whip", height="8", width="50", font="Arial",)
ChatBox.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(root, font=("Red",8,'bold'), text="Send", width="20", height=6,
                    bd=0, bg="PaleVioletRed3", activebackground="LightSalmon2",fg='#000000',
                    command= send )
#Create the box to enter message
EntryBox = Text(root, bd=0, bg="light sea green",width="29", height="5", font="Times")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()

In [44]:
m=send

In [45]:
m

<function __main__.send()>